# Import Packages and Data

In [44]:
import pandas as pd
import numpy as np

transcript = pd.read_pickle(r'data\02_stg\stg_transcript.pkl')
portfolio = pd.read_pickle(r'data\02_stg\stg_portfolio_featurized.pkl')
transcript.head()


,customer_id,offer_id,event,time_hrs,transaction_amount,reward
0,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,offer received,168,NaN,NaN
1,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,offer viewed,192,NaN,NaN
2,0009655768c64bdeb2e877511632db8f,NaN,transaction,228,22.16,NaN
3,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,offer received,336,NaN,NaN
4,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,offer viewed,372,NaN,NaN


# Wrangle Offers Data

In [45]:
#Drop the 'transaction' rows
offers = transcript[transcript['event'] != 'transaction']

# Drop the 'transaction_amount' column
offers = offers.drop('transaction_amount', axis=1)

df_pivot_offers = offers.pivot_table(index=['customer_id', 'offer_id'], 
                            values='time_hrs', 
                            columns='event', 
                            aggfunc='first').reset_index()

#Re order columns
df_pivot_offers = df_pivot_offers[['customer_id', 'offer_id', 'offer received', 'offer viewed', 'offer completed']]

#Rename columns
df_pivot_offers.columns = ['customer_id', 'offer_id', 'offer_received', 'offer_viewed', 'offer_completed']

#Sort columns by customer_id and offer_received to make customer segmentation easier
df_pivot_offers = df_pivot_offers.sort_values(by=['customer_id', 'offer_received'])

#Reset index
df_pivot_offers = df_pivot_offers.reset_index(drop=True)

df_pivot_offers.head()

,customer_id,offer_id,offer_received,offer_viewed,offer_completed
0,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,168.0,192.0,NaN
1,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,336.0,372.0,NaN
2,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408.0,456.0,414.0
3,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,504.0,540.0,528.0
4,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,576.0,NaN,576.0


# Calculate Time to Completion

In [46]:
def time_to_complete(row):
    # Check if the offer was viewed
    offer_viewed = pd.notnull(row['offer_viewed'])
    # Check if the offer was completed
    offer_completed = pd.notnull(row['offer_completed'])
    
    # If the offer was viewed and completed, and the time of viewing is less than or equal to the time of completion
    if offer_viewed and offer_completed and row['offer_viewed'] <= row['offer_completed']:
        # Return the time difference between completion and viewing
        return row['offer_completed'] - row['offer_viewed']
    # If the offer was not viewed but was completed
    # This is still important to calculate because we can use this to optimize the timing of push notifications
    elif offer_completed:
        # Return the time difference between completion and receiving the offer
        return row['offer_completed'] - row['offer_received']
    else:
        # If the offer was neither viewed nor completed, return NaN
        return np.nan

In [47]:
# Apply the function to the dataframe
df_pivot_offers['time_to_completion'] = df_pivot_offers.apply(time_to_complete, axis=1)
df_pivot_offers.head()

,customer_id,offer_id,offer_received,offer_viewed,offer_completed,time_to_completion
0,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,168.0,192.0,NaN,NaN
1,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,336.0,372.0,NaN,NaN
2,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408.0,456.0,414.0,6.0
3,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,504.0,540.0,528.0,24.0
4,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,576.0,NaN,576.0,0.0


# Join Offer Information to Offers

In [48]:
portfolio.head()

,offer_id,difficulty,reward,duration_days,duration_hrs,email,mobile,social,web,is_bogo,is_discount,is_informational
0,ae264e3637204a6fb9bb56bc8210ddfd,10,10,7,168,1,1,1,0,1,0,0
1,4d5c57ea9a6940dd891ad53e9dbe8da0,10,10,5,120,1,1,1,1,1,0,0
2,3f207df678b143eea3cee63160fa8bed,0,0,4,96,1,1,0,1,0,0,1
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,5,7,168,1,1,0,1,1,0,0
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,20,5,10,240,1,0,0,1,0,1,0


In [49]:
# Drop 'duration_days' column
portfolio = portfolio.drop('duration_days', axis=1)

# Merge df_merged with the subset of portfolio
df_merged = pd.merge(df_pivot_offers, portfolio, on='offer_id', how='left')

# Created 'expiration' column
df_merged['expiration'] = df_merged['offer_received'] + df_merged['duration_hrs']

df_merged.head()

,customer_id,offer_id,offer_received,offer_viewed,offer_completed,time_to_completion,difficulty,reward,duration_hrs,email,mobile,social,web,is_bogo,is_discount,is_informational,expiration
0,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,168.0,192.0,NaN,NaN,0,0,72,1,1,1,0,0,0,1,240.0
1,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,336.0,372.0,NaN,NaN,0,0,96,1,1,0,1,0,0,1,432.0
2,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408.0,456.0,414.0,6.0,5,5,120,1,1,1,1,1,0,0,528.0
3,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,504.0,540.0,528.0,24.0,10,2,240,1,1,1,1,0,1,0,744.0
4,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,576.0,NaN,576.0,0.0,10,2,168,1,1,0,1,0,1,0,744.0


# Summarize Data

In [50]:
# Calculate the number of unique customers
num_customers = df_merged['customer_id'].nunique()
print(f'There are {num_customers} unique customers in the dataset.')

# Calculate the length of the experiment
end_hrs = offers['time_hrs'].max()
days, hours = divmod(end_hrs, 24)
print(f'The experiment lasted {days} days and {hours} hours.')

There are 16994 unique customers in the dataset.
The experiment lasted 29 days and 18 hours.


In [61]:
### I need to modify this so that we're only looking at the deals completed after the offer was viewed
# Calculate the number of offers received, viewed, and completed per customer
offers_per_cust = df_merged.groupby('customer_id').agg(
    num_offers_received=('offer_id', 'count'),
    num_offers_viewed=('offer_viewed', 'count'),
    num_offers_completed=('offer_completed', 'count')
).reset_index().sort_values('num_offers_received', ascending=False)

offers_per_cust.to_csv(r'data\04_fct\fct_offers_per_customer.csv', index=False)
offers_per_cust.head()

,customer_id,num_offers_received,num_offers_viewed,num_offers_completed
14018,d24aac66f5404bc48728a207cee8b098,6,6,4
1652,199a0cb9068941d5a520b891e0d1e9ae,6,6,4
11554,ac76ca1ff69c456dad050ae58ad82647,6,4,2
8465,7f495962e50f40b1b76393c7c5343135,6,5,3
16126,f2c52d3d820c466aa70b414bf1b5510f,6,6,4


In [65]:
# Calculate the number of customers who received a certain number of offers
customer_count = offers_per_cust.groupby('num_offers_received')['customer_id'].nunique().reset_index(name='num_customers')

#Calculate the percentage of customers who received a certain number of offers
customer_count['percent_customers'] = round(customer_count['num_customers'] / num_customers * 100, 2)

# Save the dataframe to a CSV file
customer_count.to_csv(r'data\04_fct\fct_customers_per_num_offers.csv', index=False)
customer_count

,num_offers_received,num_customers,percent_customers
0,1,172,1.01
1,2,1599,9.41
2,3,5038,29.65
3,4,6563,38.62
4,5,3180,18.71
5,6,442,2.60


In [58]:
# filter df_merged on customer_id 'd24aac66f5404bc48728a207cee8b098'
customer = df_merged[df_merged['customer_id'] == 'd24aac66f5404bc48728a207cee8b098']
customer.head()

,customer_id,offer_id,offer_received,offer_viewed,offer_completed,time_to_completion,difficulty,reward,duration_hrs,email,mobile,social,web,is_bogo,is_discount,is_informational,expiration
52211,d24aac66f5404bc48728a207cee8b098,5a8bc65990b245e5a138643cd4eb9837,0.0,0.0,NaN,NaN,0,0,72,1,1,1,0,0,0,1,72.0
52212,d24aac66f5404bc48728a207cee8b098,4d5c57ea9a6940dd891ad53e9dbe8da0,168.0,168.0,180.0,12.0,10,10,120,1,1,1,1,1,0,0,288.0
52213,d24aac66f5404bc48728a207cee8b098,9b98b8c7a33c4b65b9aebfe6a799e6d9,336.0,354.0,360.0,6.0,5,5,168,1,1,0,1,1,0,0,504.0
52214,d24aac66f5404bc48728a207cee8b098,2906b810c7d4411798c6938adc9daaa5,408.0,408.0,438.0,30.0,10,2,168,1,1,0,1,0,1,0,576.0
52215,d24aac66f5404bc48728a207cee8b098,ae264e3637204a6fb9bb56bc8210ddfd,504.0,504.0,504.0,0.0,10,10,168,1,1,1,0,1,0,0,672.0
